In [17]:
import monai
import torch

image_dir_no_patho = ["./data/ATLAS/splits_healthy_54_sample/Atlas_healthy.csv"]
image_dir_patho_reference = ["./data/ATLAS/splits_reference_test_unhealthy_54_samples/atlas_test_png.csv"]
image_dir_patho_reference_same_atlas = ["./data/ATLAS/splits_over_1_stratified_all_splits/atlas_test_png.csv"]


from torch.utils.data import DataLoader
import sys
import os

os.chdir('/home/malek/mock/autoDDPM')
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))


# Get the parent directory
# Add the parent directory to the system path
sys.path.append(dir_path)
print(dir_path)
from data.loaders.ixi_loader import mask_preprocessing_loader, AtlasLoader, IXILoaderHarmonize, AtlasLoaderHarmonize

from PIL import Image
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"

dataset_no_patho = AtlasLoader(
            image_dir_no_patho,
            target_size=(128, 128),
            test=False,
        )

loader_no_patho = DataLoader(
    dataset_no_patho, batch_size=54, shuffle=False, drop_last=False, pin_memory=True
)

dataset_patho_referemce = AtlasLoader(
            image_dir_patho_reference,
            target_size=(128, 128),
            test=False,
        )

loader_patho_reference = DataLoader(
    dataset_patho_referemce, batch_size=54, shuffle=False, drop_last=False, pin_memory=True
)

dataset_patho_referemce_same_atlas = AtlasLoader(
            image_dir_patho_reference_same_atlas,
            target_size=(128, 128),
            test=False,
        )

loader_patho_reference_same_atlas = DataLoader(
    dataset_patho_referemce_same_atlas, batch_size=54, shuffle=False, drop_last=False, pin_memory=True
)




/home/malek/mock


In [2]:
from generative.metrics import FIDMetric, MMDMetric, MultiScaleSSIMMetric, SSIMMetric

radnet = torch.hub.load("Warvito/radimagenet-models:main", model="radimagenet_resnet50",verbose=True)

radnet.to(device)
radnet.eval()

Using cache found in /home/malek/.cache/torch/hub/Warvito_radimagenet-models_main


ResNet50(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.01, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1.001e-05, momentum=0.99, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm

In [6]:
def subtract_mean(x: torch.Tensor) -> torch.Tensor:
    mean = [0.406, 0.456, 0.485]
    x[:, 0, :, :] -= mean[0]
    x[:, 1, :, :] -= mean[1]
    x[:, 2, :, :] -= mean[2]
    return x


def spatial_average(x: torch.Tensor, keepdim: bool = True) -> torch.Tensor:
    return x.mean([2, 3], keepdim=keepdim)


def get_features(image):
    # If input has just 1 channel, repeat channel to have 3 channels
    if image.shape[1]:
        image = image.repeat(1, 3, 1, 1)

    # Change order from 'RGB' to 'BGR'
    image = image[:, [2, 1, 0], ...]

    # Subtract mean used during training
    image = subtract_mean(image)

    # Get model outputs
    with torch.no_grad():
        feature_image = radnet.forward(image)
        # flattens the image spatially
        feature_image = spatial_average(feature_image, keepdim=False)

    return feature_image


In [19]:
for (data_1,data_2,data_3) in zip(loader_no_patho,loader_patho_reference,loader_patho_reference_same_atlas):
    images_no_patho = data_1[0].to(device)
    images_patho_reference = data_2[0].to(device)
    images_patho_reference_same_atlas = data_3[0].to(device)
    
    
    print("shape of images_no_patho",images_no_patho.shape)



shape of images_no_patho torch.Size([54, 1, 128, 128])


# Compute SSIM

In [11]:
ssim = SSIMMetric(spatial_dims=2, data_range=1.0, kernel_size=4)
ms_ssim = MultiScaleSSIMMetric(spatial_dims=2, data_range=1.0, kernel_size=4)



In [15]:
ssim_result = ssim(images_no_patho, images_patho_reference)
ms_ssim_result = ms_ssim(images_no_patho, images_patho_reference)

print(f"SSIM Metric between pathological (reference) and non pathological Atlas subsets: {ssim_result.mean():.7f} +- {ssim_result.std():.7f}")
print(f"MS-SSIM Metric between pathological (reference) and non pathological Atlas subsets (reference): {ms_ssim_result.mean():.7f} +- {ms_ssim_result.std():.7f}")

SSIM Metric between pathological (reference) and non pathological Atlas subsets: 0.4895904 +- 0.0621779
MS-SSIM Metric between pathological (reference) and non pathological Atlas subsets (reference): 0.6403977 +- 0.0575180


In [20]:
ssim_result = ssim(images_no_patho, images_patho_reference_same_atlas)
ms_ssim_result = ms_ssim(images_no_patho, images_patho_reference_same_atlas)

print(f"SSIM Metric between pathological (reference) and non pathological Atlas subsets: {ssim_result.mean():.7f} +- {ssim_result.std():.7f}")
print(f"MS-SSIM Metric between pathological (reference) and non pathological Atlas subsets (same atlas): {ms_ssim_result.mean():.7f} +- {ms_ssim_result.std():.7f}")


SSIM Metric between pathological (reference) and non pathological Atlas subsets: 0.4580999 +- 0.0717665
MS-SSIM Metric between pathological (reference) and non pathological Atlas subsets (same atlas): 0.6345845 +- 0.0574853


In [16]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_no_patho)
real_features.append(real_eval_feats)

# Get the features for the synthetic data
synth_eval_feats = get_features(images_patho_reference)
synth_features.append(synth_eval_feats)

synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)

print('FID score between pathological (reference) and non pathological Atlas subsets')
print(f"FID Score: {fid_res.item():.2f}")

FID score between pathological (reference) and non pathological Atlas subsets
FID Score: 6.30


In [21]:
synth_features = []
real_features = []

real_eval_feats = get_features(images_no_patho)
real_features.append(real_eval_feats)

# Get the features for the synthetic data
synth_eval_feats = get_features(images_patho_reference_same_atlas)
synth_features.append(synth_eval_feats)

synth_features = torch.vstack(synth_features)
real_features = torch.vstack(real_features)

fid = FIDMetric()
fid_res = fid(synth_features, real_features)

print('FID score between pathological (reference same atlas split) and non pathological Atlas subsets')
print(f"FID Score: {fid_res.item():.2f}")

FID score between pathological (reference same atlas split) and non pathological Atlas subsets
FID Score: 7.08
